# seeing how bad the toy is

In [ ]:
import numpy as np
from numpy import ndarray
from numpy.random import default_rng

from scipy import sparse as sp
from matplotlib import pyplot as plt

import networkx as nx

A proc to draw value paths in the toy bnb tree

In [ ]:
from toybnb.tree import build_optresult


def path_to_root(T: nx.DiGraph, node: int) -> list[int, ...]:
    """Ascent path to the root form the node."""
    path = [node]
    while T.pred[node]:
        node = next(iter(T.pred[node]))
        path.append(node)

    return path


def path_to_best(T: nx.DiGraph) -> list[int, ...]:
    """Descend from the root down the tree to the best solution."""
    node = T.graph["root"]
    best = nx.get_node_attributes(T, "best")
    path, inc = [node], T.graph["incumbent"]
    while T[node]:
        for child in T[node]:
            if best.get(child) is inc:
                node = child
                break
        else:
            break
        path.append(node)

    return path


def get_value_paths(T: nx.DiGraph) -> dict[int, list]:
    """Extract the value function from the tree."""
    best = nx.get_node_attributes(T, "best")
    leaves = [n for n in T if not T[n]]

    vfn, nil = {}, build_optresult()
    for leaf in leaves:
        vf = vfn[leaf] = []
        path = path_to_root(T, leaf)
        for j, n in enumerate(reversed(path)):
            b = best.get(n, nil) or nil
            vf.append((j, b.fun))

    return vfn

A proc to draw toy bnb trees (or from scip Tracer)

In [ ]:
from toybnb.tree import Status
from matplotlib.collections import PatchCollection


def draw_tree(T, pos=None, ax=None, nodelist=None):
    ax = ax if ax is None else plt.gca()

    st = nx.get_node_attributes(T, "status")
    best = nx.get_node_attributes(T, "best")

    # draw only open, feasible, or infeasible nodes only
    allowed = {Status.OPEN, Status.INFEASIBLE, Status.FEASIBLE, Status.FATHOMED}
    # INFEASIBLE, PRUNED, FEASIBLE -- fathomed node
    # CLOSED -- no branching options, OPEN -- node yet to be explored
    if nodelist is None:
        nodelist = set([n for n, s in st.items() if s in allowed])
        nodelist.update(path_to_best(T))

    nodelist = list(nodelist)

    # assign colors
    nodecolor, inc = [], T.graph["incumbent"]
    for n in nodelist:
        if inc is best.get(n):
            nodecolor.append("C3")
            continue

        match st[n]:
            case Status.FEASIBLE:
                nodecolor.append("C3")
            case Status.INFEASIBLE:
                nodecolor.append("C2")
            case Status.OPEN:
                nodecolor.append("fuchsia")
            case Status.FATHOMED:
                nodecolor.append("C4")
            case _:
                nodecolor.append("#0008")

    # plot the branching edges as lightly as possible
    col = PatchCollection(
        nx.draw_networkx_edges(
            T,
            pos,
            ax=ax,
            width=0.5,
            node_size=0,
            arrows=None,
            arrowsize=0,
            arrowstyle="-",
            edge_color="#0004",
        )
    )
    col.set_zorder(-10)

    col = nx.draw_networkx_nodes(
        T,
        pos,
        ax=ax,
        node_size=1,
        node_shape="s",
        nodelist=nodelist,
        node_color=nodecolor,
    )
    col.set_zorder(10)

    ax.set_frame_on(False)

Invert out bnb solver at the branch variable selection, while still using the depth-first node selector.

In [ ]:
from toybnb.milp import MILP
from toybnb import search as bnb
from toybnb.coro import Coroutine


def inverted_search(p: MILP) -> ...:
    """Inverted variable branching as generator"""

    def branchrule(G: nx.DiGraph, node: int) -> int:
        return co.co_yield((G, node))

    args = p, bnb.nodesel_dfs, branchrule
    co = Coroutine(bnb.search, args=args)
    return iter(co)


def branching(p: MILP) -> tuple[nx.DiGraph, int]:
    """Branching that allows each node to be visited only once"""
    it, visited, var = inverted_search(p), set(), None
    try:
        while True:
            G, node = it.send(var)
            while node in visited:
                G, node = it.throw(IndexError)

            visited.add(node)
            var = yield G, node
            # it.gi_frame.f_locals["self"].co_is_suspended

    except StopIteration as e:
        return e.value, None


def send(it: iter, value: ...) -> ...:
    """Send a value to the generator and get a value from it in return"""
    try:
        return it.send(value), False
    except StopIteration as e:
        return e.value, True


class GeneratorEnv:
    """A wrapper to make generators into envs."""

    def reset(self, it: iter) -> ...:
        self.it = iter(it)
        return send(self.it, None)

    def step(self, act: ...) -> ...:
        return send(self.it, act)

<br>

### ToyBNB tree

Generate a simple problem and try to solve it with random variable branching.

In [ ]:
from tqdm import tqdm
from toybnb.milp import generate

# it = generate(1500, 1200, 5, seed=53912)
it = generate(50, 38, 10, seed=1458)
p = next(it)  # x = it.gi_frame.f_locals["p"]

with tqdm(ncols=70) as pb:
    rng = default_rng()
    env = GeneratorEnv()
    (T, node), fin = env.reset(branching(p))
    while not fin:
        pb.update(1)
        # pick a random index
        dt = T.nodes[node]
        mask = np.unpackbits(dt["mask"], count=dt["p"].n, bitorder="little")
        var = rng.choice(np.flatnonzero(mask))

        # branch
        (T, node), fin = env.step(var)
    assert node is None

Plot the primal and dual bound history

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 1, dpi=300)
nn, pv, dv = map(np.array, zip(*T.graph["track"]))
ax.plot(dv)
ax.plot(pv)

Draw the value paths

In [ ]:
vfn = get_value_paths(T)

fig, ax = plt.subplots(1, 1, figsize=(10, 4), dpi=120)
for leaf, vf in vfn.items():
    ax.plot(*zip(*vf), label=leaf, c="C0", alpha=0.24)

Draw the search tree

In [ ]:
pos = nx.nx_agraph.graphviz_layout(T, prog="dot", args="")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
draw_tree(T, pos, ax)

In [ ]:
T.nodes[0]["best"]

<br>

### Strong branching

The lp-gains $\Delta_\pm$ are computed based on the left and right relaxation of the original problem. Let the original feasibility set and its continuous relaxation be, respecitvely,
$$
\begin{align}
S
    &= \bigl\{
        x \in \mathbb{Z}^m \times \mathbb{R}^{n-m}
        \colon A x \leq b
        \,, x \in [l, u] 
    \bigr\}
    \,, \\
\breve{S}
    &= \bigl\{
        x \in \mathbb{R}^n
        \colon A x \leq b
        \,, x \in [l, u] 
    \bigr\}
    \,.
\end{align}
$$
The optimization problem
$$
\breve{f}
    = \min_x \{
        c^\top x\colon x \in \breve{S}
    \}
    \,, $$
offers a lower bound on the achievable value of the objective in the integer-feasibile region $S$. If we happen to have a candidate (incumbent) $x_* \notin S$ with $f^* = c^\top x_*$, but integer-feasible in the __original problem's domain__, then $f^* < \breve{f}$ is a certificate that no integer-feasible solutions better than $x$ can be found in $\breve{S}$. This means that $S \subseteq \breve{S}$ maybe excluded from search.

If $\breve{f} \leq f^*$ and $\breve{x} \notin S$, there is some $j=1..m$ such that $\breve{x}^j \notin \mathbb{Z}$. Since it would be impossible for any integer-feasible solution $x \in S$ to have $
    x^j \in \bigl(
        \lfloor \breve{x}^j \rfloor,
        \lceil \breve{x}^j \rceil
    \bigr)
$, it becomes possible to split the original feasibility set $S$ in two non-adjacent subsets:
$$
    \underbrace{
        \bigl\{
            % x \colon
            x^j \leq \lfloor \breve{x}^j \rfloor
        \bigr\}  % \times \mathbb{R}^{n-1}
    }_{R^j_-}
    \uplus
    \underbrace{
        \bigl\{
            % x \colon
            \lceil \breve{x}^j \rceil \geq x^j
        \bigr\}  % \times \mathbb{R}^{n-1}
    }_{R^j_+}
    \,.
$$
Note that it is sufficient to split the region $S$ with respect to one variable only, since every other split is guaranteed by integer-feasiblity __not__ to containt a feasible solution in the excluded region of the $j$-th variable.

The sub-regions $
    S^j_\pm = S \cap R^j_\pm
$ bring about new lower bounds $
\breve{f}^j_\pm
    = \min_x \{
        c^\top x\colon x \in \breve{S}^j_\pm
    \}
$. The new sub-problems are obtained from the original by modifying the bounds of the $j$-th variable: $
    \bigl[l^j, \lfloor \breve{x}^j \rfloor\bigr]
$ and $
    \bigl[\lceil \breve{x}^j \rceil, u^j\bigr]
$. The amount of the objective value, by which each bound is thighened, is the _gain_:
$$
\Delta^j_\pm
    = \breve{f}^j_\pm - \breve{f}
    \geq 0
    \,. $$

In [ ]:
from toybnb.tree import lpsolve, OptimizeResult, split


def get_lp_gains(
    p: MILP, lp: OptimizeResult, mask: ndarray
) -> tuple[ndarray, ndarray, int]:
    """Compute LP branching gains for each candidate variable in the binary mask."""
    lpiter = 0

    # split by the variable, solve LPs, and record dual gains
    gains = np.full((p.n, 2), np.nan)
    for j in np.flatnonzero(mask):
        lp_lo, lp_up = map(lpsolve, split(p, j, lp.x[j]))
        gains[j] = lp_lo.fun - lp.fun, lp_up.fun - lp.fun
        lpiter += lp_lo.nit + lp_up.nit

    return gains.clip(0), lpiter

Scoring functions
* additive $
    s_j = \mu \max\{\Delta^j_-, \Delta^j_+\}
        + (1 - \mu) \min\{\Delta^j_-, \Delta^j_+\}
$
* multiplicative $
    s_j = \Delta^j_- \Delta^j_+
$ -- the default scorefunc in SCIP

In [ ]:
with tqdm(ncols=70) as pb:
    n_lpit = 0
    env = GeneratorEnv()
    (T, node), fin = env.reset(branching(p))
    while not fin:
        pb.update(1)

        dt = T.nodes[node]

        # get the up-lo branching gains
        mask = np.unpackbits(dt["mask"], count=dt["p"].n, bitorder="little")
        gains, nit = get_lp_gains(dt["p"], dt["lp"], mask)
        n_lpit += nit

        # scores = mu * gains.max(-1) + (1 - mu) * gains.min(-1)
        scores = gains[:, 0] * gains[:, 1]

        var = np.nanargmax(scores)

        # branch
        (T, node), fin = env.step(var)
    assert node is None

Plot the track, the value paths, and the tree

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 1, dpi=300)
nn, pv, dv = map(np.array, zip(*T.graph["track"]))
ax.plot(dv)
ax.plot(pv)

In [ ]:
vfn = get_value_paths(T)

fig, ax = plt.subplots(1, 1, figsize=(10, 4), dpi=120)
for leaf, vf in vfn.items():
    ax.plot(*zip(*vf), label=leaf, c="C0", alpha=0.24)

In [ ]:
pos = nx.nx_agraph.graphviz_layout(T, prog="dot", args="")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
draw_tree(T, pos, ax)
fig.savefig("sb_tree.pdf")

<br>

### SCIP

We've got our own custom [build](https://github.com/ivannz/ecole) of ecole.

In [ ]:
import os, sys

sys.path.append(
    os.path.abspath("../../../repos_with_rl/copt/ecole/build/cmake/python/ecole")
)

import ecole as ec

Let's use the tree tracer for SCIP

In [ ]:
# from toybnb.scip.tracer import TracedBranching, Branching
from toybnb.scip.tracer import TracedBranching, Branching


scip_params = {
    "branching/scorefunc": "p",
    "randomization/permuteconss": False,
    "randomization/lpseed": 27372012,
    "branching/random/seed": 41,
    "branching/relpscost/startrandseed": 5,
}


env = TracedBranching(
    observation_function=(
        ec.observation.NodeBipartite(),
        ec.observation.StrongBranchingScores(),
    ),
    scip_params=scip_params,
)

Convert toy's MILP into SCIP's model

In [ ]:
from toybnb.scip import to_scip

fin = True
it = ec.instance.CombinatorialAuctionGenerator()
while fin:
    mod = ec.scip.Model.from_pyscipopt(to_scip(p))
    # mod = next(it)
    obs, act, rew, fin, nfo = env.reset(mod)
    break

assert not fin

SCIP is so much better than the thing we implemented, so there is little hope to see really deep trees on the problems, that toy can handle.

In [ ]:
from toybnb.scip.scip import from_scip_lp

with tqdm(ncols=70) as pb:
    rng = default_rng()

    obs, act, rew, fin, nfo = env.reset(mod)
    m = env.model.as_pyscipopt()

    while not fin:
        pb.update(1)
        bi, sb = obs

        # read the local MILP at the focus node
        # x: MILP = from_scip_lp(env.model.as_pyscipopt())

        # pick a random index
        var = rng.choice(act)  # obs[act], act

        # or use sb scores
        var = act[sb[act].argmax(-1)]
        # branch
        obs, act, rew, fin, nfo = env.step(var)

        m = env.model.as_pyscipopt()

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 1, dpi=300)
nn, pv, dv = map(np.array, zip(*env.tracer.trace_))
ax.plot(dv)
ax.plot(pv)

In [ ]:
pos = nx.nx_agraph.graphviz_layout(env.tracer.T, prog="dot", args="")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
draw_tree(env.tracer.T, pos, ax, nodelist=None)  # , nodelist=T.graph["fathomed"])

<br>

# Trunk

The bipartite observations from ecole

In [ ]:
# 0 obj 1:4 type onehot 5 lb 6 ub 7 reducedcost
# 8 lp-val 9 lp-frac 10 at lb 11 at ub 12 age
# 13 incumbent-val 14 avg-incumbent-val 15:19 is_basis onehot
bi.variable_features[0]

In [ ]:
shape = len(bi.row_features), len(bi.variable_features)
tab = sp.coo_array((bi.edge_features.values, bi.edge_features.indices), shape)
bi.row_features.shape
bi.variable_features[0]

In [ ]:
plt.imshow(tab.toarray())

In [ ]:
cos = tab.dot(tab.T).toarray()
plt.imshow(abs(cos))

In [ ]:
from scipy import sparse as sp

A_ub = x.A_ub
n = sp.linalg.norm(A_ub, axis=-1, ord=2)
U = sp.diags(np.where(n > 0, 1 / n, 1.0)) * A_ub
cos = U.dot(U.T).toarray()

In [ ]:
plt.imshow(abs(cos))

In [ ]:
abs(cos).round(3)

The ecole env and our env diverge

In [ ]:
from toybnb.scip.scip import from_scip, from_scip_lp

mod = ec.scip.Model.from_pyscipopt(to_scip(p))
mod.transform_prob()
mod.presolve()
(bi, obs), act, rew, fin, nfo = env.reset(mod)

x = from_scip_lp(env.model.as_pyscipopt())  # x = it.gi_frame.f_locals["p"]
it = branching(x)

n_it = 0
G, node = it.send(None)
while True:
    n_it += 1

    mask = np.unpackbits(dt["mask"], count=dt["p"].n, bitorder="little")
    if not np.isin(act, np.flatnonzero(mask)).all():
        break

    gains, nit = get_lp_gains(dt["p"], dt["lp"], mask)

    scores = gains[:, 0] * gains[:, 1]
    var = np.nanargmax(scores)

    (bi, obs), act, rew, fin, nfo = env.step(var)
    G, node = it.send(var)

    # now scip and toy may diverge
    x: MILP = from_scip_lp(env.model.as_pyscipopt())

print(n_it)

<br>